In [3]:
class projeto_envio():
    
    def __init__(self):
        self.leitura_base()
        self.transform_data()
        self.documento()


    def leitura_base(self):    
        #Importação Biblioteca
        import pandas as pd
        
        #Path/Caminho do arquivo
        path_arquivo = '../database/Database_client.csv'
        
        #Leitura do arquivo .csv
        def leitura_arquivo(path, separador=';', idioma='latin-1'):
            
            arquivo = pd.read_csv(path, sep=separador,encoding=idioma)
            return arquivo
        #Arquivo "Lido"
        self.arquivo = leitura_arquivo(path=path_arquivo, idioma='latin-1')
         
    def transform_data(self):       
        #Tranformando a coluna "Telefone" no formato (00) 0000-0000,  Digamos que todos os DDD fossem iguais a 24, mas pode inserir outro valor na variavel DDD, que funcionara da mesma maneira
        def transformar_numero(dataframe,DDD='', coluna_a_transformar=''):
            dataframe[coluna_a_transformar] = f'({DDD})' + dataframe[coluna_a_transformar].astype(str)
            dataframe[coluna_a_transformar] = dataframe[coluna_a_transformar].apply(lambda x: f'{x[:9]}-{x[9:]}')
            return dataframe
        
        #Transformando a coluna "Data da Última Compra" em datetime
        def transformar_data(dataframe, formato='%d/%m/%Y', coluna_a_transformar=''):
            from pandas import to_datetime
            dataframe[coluna_a_transformar] = to_datetime(dataframe[coluna_a_transformar], format=formato)  
            return
            
        arquivo_transformado = self.arquivo.copy()
        transformar_numero(dataframe=arquivo_transformado, DDD=24, coluna_a_transformar='Telefone')
        transformar_data(dataframe=arquivo_transformado,coluna_a_transformar='Data da Última Compra')
        
        self.arquivo_transformado = arquivo_transformado.copy()
        
    def documento(self):
        #Extraindo dados dos clientes
        for i in self.arquivo_transformado:
            print(i)
        
        

In [4]:
x = projeto_envio()

ID
Nome
Forma de Pagamento
E-mail
Telefone
Data da Última Compra
Endereço
